In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import requests
import zipfile

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

print("TensorFlow version:", tf.__version__)
zip_file = "/archive.zip"  # 修改为你的文件路径
extract_folder = "/content/extracted_folder"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("解压完成！")

# FER2013 中共有 7 个类别，顺序：Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# 修改为你在 Colab 上解压数据集的路径（训练集与测试集）
train_dir = "/content/extracted_folder/train"   # 请确保此路径正确
val_dir   = "/content/extracted_folder/test"      # 请确保此路径正确

img_width, img_height = 48, 48
batch_size = 64

# 数据增强与生成器（注意设置 color_mode 为 'grayscale'）
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="sparse",  # 标签将以整数形式输出
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="sparse",
    shuffle=False
)

def build_model(input_shape=(48,48,1), num_classes=7):
    inputs = Input(shape=input_shape)
    # Block 1
    x = Conv2D(64, (3,3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(64, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.25)(x)
    # Block 2
    x = Conv2D(128, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(128, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.25)(x)
    # Block 3
    x = Conv2D(256, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(256, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation="softmax")(x)
    model = Model(inputs, outputs)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

model = build_model()

# 修改回调函数：移除 EarlyStopping 以确保完整运行 300 个 epoch
callbacks = [
    ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=10, min_lr=1e-5, verbose=1),
    ModelCheckpoint("best_model.h5", monitor="val_accuracy", save_best_only=True, verbose=1)
]

history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = val_generator,
    validation_steps = val_generator.samples // batch_size,
    epochs = 300,
    callbacks = callbacks,
    verbose = 2
)

model.save("facial_expression_model.h5")

# 评估模型
val_generator.reset()
loss, acc = model.evaluate(val_generator, verbose=0)
print(f"Validation Accuracy: {acc*100:.2f}%")
val_generator.reset()
y_pred = np.argmax(model.predict(val_generator), axis=1)
y_true = val_generator.classes
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=list(train_generator.class_indices.keys())))
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10,8))
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
ticks = np.arange(len(train_generator.class_indices))
plt.xticks(ticks, list(train_generator.class_indices.keys()), rotation=45)
plt.yticks(ticks, list(train_generator.class_indices.keys()))
plt.tight_layout()
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.savefig("confusion_matrix.png")
plt.show()

# 注意：Colab 无法直接使用本地摄像头，实时识别部分在 Colab 上通常无法正常运行。
# 如果你需要在本地运行实时表情识别，请在本地环境中运行以下代码：

def real_time_expression_recognition(model):
    cascade_file = "haarcascade_frontalface_default.xml"
    if not os.path.exists(cascade_file):
        url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
        r = requests.get(url)
        if r.status_code == 200:
            with open(cascade_file, "wb") as f:
                f.write(r.content)
            print("Haar Cascade downloaded.")
        else:
            print("Failed to download Haar Cascade.")
            return
    face_cascade = cv2.CascadeClassifier(cascade_file)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open webcam.")
        return
    print("Starting real-time expression recognition. Press 'q' to exit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
        for (x, y, w, h) in faces:
            face_roi = gray[y:y+h, x:x+w]
            try:
                face_roi = cv2.resize(face_roi, (48,48))
            except Exception:
                continue
            face_roi = face_roi.astype("float32") / 255.0
            face_roi = np.expand_dims(face_roi, axis=-1)
            face_roi = np.expand_dims(face_roi, axis=0)
            pred = model.predict(face_roi)
            label_idx = np.argmax(pred)
            label = list(train_generator.class_indices.keys())[label_idx]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255,0,0), 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        cv2.imshow("Real-time Expression Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

real_time_expression_recognition(model)

TensorFlow version: 2.18.0
解压完成！
Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/300


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from -inf to 0.34584, saving model to best_model.h5


448/448 - 49s - 110ms/step - accuracy: 0.2804 - loss: 1.9150 - val_accuracy: 0.3458 - val_loss: 1.7049 - learning_rate: 1.0000e-03
Epoch 2/300


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


...

448/448 - 2s - 5ms/step - accuracy: 0.6406 - loss: 0.9057 - val_accuracy: 0.6858 - val_loss: 0.9034 - learning_rate: 1.0000e-05
Epoch 141/300

Epoch 141: val_accuracy did not improve from 0.68583
448/448 - 25s - 55ms/step - accuracy: 0.7336 - loss: 0.7152 - val_accuracy: 0.6846 - val_loss: 0.9027 - learning_rate: 1.0000e-05
Epoch 142/300

Epoch 142: val_accuracy did not improve from 0.68583
448/448 - 2s - 5ms/step - accuracy: 0.7656 - loss: 0.6963 - val_accuracy: 0.6850 - val_loss: 0.9027 - learning_rate: 1.0000e-05
Epoch 143/300

Epoch 143: val_accuracy did not improve from 0.68583
448/448 - 42s - 93ms/step - accuracy: 0.7324 - loss: 0.7145 - val_accuracy: 0.6840 - val_loss: 0.9032 - learning_rate: 1.0000e-05
Epoch 144/300

Epoch 144: val_accuracy did not improve from 0.68583
448/448 - 4s - 10ms/step - accuracy: 0.7656 - loss: 0.8101 - val_accuracy: 0.6842 - val_loss: 0.9029 - learning_rate: 1.0000e-05
Epoch 145/300

Epoch 145: val_accuracy did not improve from 0.68583
448/448 - 25s -

448/448 - 41s - 92ms/step - accuracy: 0.7333 - loss: 0.7070 - val_accuracy: 0.6861 - val_loss: 0.9046 - learning_rate: 1.0000e-05
Epoch 150/300

Epoch 150: val_accuracy did not improve from 0.68610
448/448 - 2s - 5ms/step - accuracy: 0.5938 - loss: 0.9874 - val_accuracy: 0.6858 - val_loss: 0.9047 - learning_rate: 1.0000e-05
Epoch 151/300

Epoch 151: val_accuracy did not improve from 0.68610
448/448 - 41s - 91ms/step - accuracy: 0.7319 - loss: 0.7165 - val_accuracy: 0.6843 - val_loss: 0.9030 - learning_rate: 1.0000e-05
Epoch 152/300

Epoch 152: val_accuracy did not improve from 0.68610
448/448 - 2s - 5ms/step - accuracy: 0.8281 - loss: 0.5464 - val_accuracy: 0.6844 - val_loss: 0.9030 - learning_rate: 1.0000e-05
Epoch 153/300

Epoch 153: val_accuracy did not improve from 0.68610
448/448 - 82s - 183ms/step - accuracy: 0.7346 - loss: 0.7165 - val_accuracy: 0.6846 - val_loss: 0.9021 - learning_rate: 1.0000e-05
Epoch 154/300

Epoch 154: val_accuracy did not improve from 0.68610
448/448 - 2s 

448/448 - 42s - 93ms/step - accuracy: 0.7330 - loss: 0.7116 - val_accuracy: 0.6865 - val_loss: 0.9045 - learning_rate: 1.0000e-05
Epoch 168/300

Epoch 168: val_accuracy did not improve from 0.68652
448/448 - 4s - 10ms/step - accuracy: 0.6250 - loss: 0.8632 - val_accuracy: 0.6865 - val_loss: 0.9048 - learning_rate: 1.0000e-05
Epoch 169/300

Epoch 169: val_accuracy did not improve from 0.68652
448/448 - 26s - 58ms/step - accuracy: 0.7331 - loss: 0.7125 - val_accuracy: 0.6855 - val_loss: 0.9044 - learning_rate: 1.0000e-05
Epoch 170/300

Epoch 170: val_accuracy did not improve from 0.68652
448/448 - 2s - 5ms/step - accuracy: 0.7812 - loss: 0.5991 - val_accuracy: 0.6855 - val_loss: 0.9043 - learning_rate: 1.0000e-05
Epoch 171/300

Epoch 171: val_accuracy did not improve from 0.68652
448/448 - 41s - 91ms/step - accuracy: 0.7350 - loss: 0.7058 - val_accuracy: 0.6847 - val_loss: 0.9051 - learning_rate: 1.0000e-05
Epoch 172/300

Epoch 172: val_accuracy did not improve from 0.68652
448/448 - 2s 

448/448 - 66s - 148ms/step - accuracy: 0.7381 - loss: 0.7047 - val_accuracy: 0.6868 - val_loss: 0.9055 - learning_rate: 1.0000e-05
Epoch 194/300

Epoch 194: val_accuracy did not improve from 0.68680
448/448 - 3s - 6ms/step - accuracy: 0.6094 - loss: 0.9792 - val_accuracy: 0.6867 - val_loss: 0.9054 - learning_rate: 1.0000e-05
Epoch 195/300

Epoch 195: val_accuracy did not improve from 0.68680
448/448 - 38s - 85ms/step - accuracy: 0.7363 - loss: 0.7061 - val_accuracy: 0.6857 - val_loss: 0.9073 - learning_rate: 1.0000e-05
Epoch 196/300

Epoch 196: val_accuracy did not improve from 0.68680
448/448 - 2s - 5ms/step - accuracy: 0.7969 - loss: 0.5514 - val_accuracy: 0.6860 - val_loss: 0.9072 - learning_rate: 1.0000e-05
Epoch 197/300

Epoch 197: val_accuracy did not improve from 0.68680
448/448 - 41s - 91ms/step - accuracy: 0.7354 - loss: 0.7030 - val_accuracy: 0.6861 - val_loss: 0.9071 - learning_rate: 1.0000e-05
Epoch 198/300

Epoch 198: val_accuracy did not improve from 0.68680
448/448 - 2s 

448/448 - 25s - 57ms/step - accuracy: 0.7370 - loss: 0.7050 - val_accuracy: 0.6869 - val_loss: 0.9055 - learning_rate: 1.0000e-05
Epoch 210/300

Epoch 210: val_accuracy did not improve from 0.68694
448/448 - 3s - 6ms/step - accuracy: 0.7188 - loss: 0.6945 - val_accuracy: 0.6868 - val_loss: 0.9055 - learning_rate: 1.0000e-05
Epoch 211/300


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
